In [5]:
import pandas as pd
import sqlite3
import os

def process_arribo_file(filepath, encoding='ISO-8859-1'):
    """Procesa archivos de avisos de arribo eliminando metadatos y ajustando encabezados."""
    data = pd.read_csv(filepath, skiprows=2, low_memory=False, encoding=encoding)
    data.columns = data.columns.str.strip().str.replace(' ', '_').str.replace(r'[^\w\s]', '_')
    return data

def process_vms_file(filepath, encoding='ISO-8859-1'):
    """Procesa archivos VMS ajustando las columnas."""
    data = pd.read_csv(filepath, encoding=encoding)
    data.columns = data.columns.str.strip().str.replace(' ', '_').str.replace(r'[^\w\s]', '_')
    return data

def create_table_from_df(db_conn, df, table_name):
    """Crea una tabla en la base de datos basada en el DataFrame dado."""
    cursor = db_conn.cursor()
    cols = ", ".join([f"{col} TEXT" if df[col].dtype == object else f"{col} REAL" for col in df.columns])
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    cursor.execute(f"CREATE TABLE {table_name} ({cols});")
    db_conn.commit()

def load_data_to_db(db_conn, data_dir, file_type, encoding='ISO-8859-1'):
    """Carga y procesa datos desde el directorio especificado a la base de datos."""
    files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    first = True
    for filepath in files:
        if 'arribo' in file_type:
            data = process_arribo_file(filepath, encoding)
        else:
            data = process_vms_file(filepath, encoding)
        if first:
            create_table_from_df(db_conn, data, file_type)
            first = False
        data.to_sql(file_type, db_conn, if_exists='append', index=False)

def query_by_date(db_conn, table_name, start_date, end_date):
    """Realiza una consulta SQL para filtrar los datos por un rango de fechas específico."""
    query = f"SELECT * FROM {table_name} WHERE Fecha >= ? AND Fecha <= ?"
    result = pd.read_sql_query(query, db_conn, params=(start_date, end_date))
    return result

# Configuración de la conexión a la base de datos SQLite
db_conn = sqlite3.connect('fish_database.db')

In [6]:

# Rutas de los directorios
arribo_directory = 'avisos_de_arribo'
vms_directory = 'vms_homologados'

# Cargar todos los datos en la base de datos
load_data_to_db(db_conn, arribo_directory, 'Arribo')
load_data_to_db(db_conn, vms_directory, 'VMS')

# Ejemplo de uso de la función para filtrar por fecha
filtered_data = query_by_date(db_conn, 'VMS', '2020-01-01', '2020-12-31')
print(filtered_data.head())

# Cerrar la conexión a la base de datos
db_conn.close()

C:\Users\ricar\AppData\Local\Temp\ipykernel_29620\3214814772.py:15: DtypeWarning: Columns (0,1,2,3,4,7,13,14,15,17,20,21,22,23,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath, encoding=encoding)


OperationalError: near ":_1": syntax error